In [2]:
from alpaca_farm.models import reward_model
from alpaca_farm.inference.decode import load_model_and_tokenizer_for_inference 
from alpaca_farm.inference.score import score_sequences_with_huggingface_given_model
import alpaca_farm.data_preprocessor as data_preprocessor
from alpaca_farm import utils
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from rlhfutils.debug_utils import load_all_dfs, load_rm, progress_rm
from statistics import mean

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-04 18:55:46,169] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
apfrm = load_rm("../apf/models/humannew/", 0)
tok, apfrm, kwargs = apfrm

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.48s/it]
Using /data/users/prasann/.cache/torch_extensions/py39_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /data/users/prasann/.cache/torch_extensions/py39_cu117/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module cuda_kernel...


In [4]:
def adjust_apfarm(row):
    qstr = ""
    if len(row['input'])>0:
        qstr = row['instruction']+"\n\nInput: "+row['input']
    else:
        qstr = row['instruction']
    
    return "Question: " + qstr + "\n\nAnswer: " + row['output']

In [5]:
train_dataset = load_dataset("tatsu-lab/alpaca_farm", 'alpaca_human_preference')['preference']
print("initial size ", len(train_dataset))
train_dataset = train_dataset.shuffle(seed=100)
eval_dataset = train_dataset.select(range(0,500))

Found cached dataset alpaca_farm (/home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_human_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 339.32it/s]
Loading cached shuffled indices for dataset at /home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_human_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a/cache-f4eaec1c5d8e30f0.arrow


initial size  9691


In [6]:
def pair_to_single(ex, choosebest=True):
    if (ex['preference']==1)==choosebest:
        return {
            'instruction': ex['instruction'], 
            'input': ex['input'],
            'output': ex['output_1']
        }
    else:
        return {
            'instruction': ex['instruction'], 
            'input': ex['input'],
            'output': ex['output_2']
        }

In [7]:
badmaps = eval_dataset.map(lambda x: pair_to_single(x, False))
goodmaps = eval_dataset.map(lambda x: pair_to_single(x, True))

Loading cached processed dataset at /home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_human_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a/cache-e3c5187e42089dd6.arrow
Loading cached processed dataset at /home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_human_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a/cache-367e129c7153e56e.arrow


In [20]:
kwargs

{'return_all_scores': True,
 'function_to_apply': 'none',
 'batch_size': 8,
 'truncation': True}

In [8]:
def scoreouts(rows):
    seqs = [adjust_apfarm(r) for r in rows]
    print(seqs[0])
    scores = progress_rm(seqs, apfrm, kwargs)
    return scores

In [9]:
bscos = scoreouts(badmaps)
badscos = [b[0]['score'] for b in bscos]

Question: Construct a proof of the following theorem:

Input: Theorem: The sum of two odd numbers is even.

Answer: Let's suppose we have two odd numbers, a and b. Then, a + b will be a sum of two odd numbers, so it will be even. This is because the sum of two odd numbers will always be odd. But if a and b are even, they will give a sum that is mathematically even, which is simply a consequence of them both being even numbers. Thus, the theorem holds and is proven.


  0%|                                                                                                                                 | 0/32 [00:00<?, ?it/s]/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 31%|█████████████████████████████████████▌                                                                                  | 10/32 [00:29<00:56,  2.57s/it]/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
 

In [10]:
gscos = scoreouts(goodmaps)
goodscos = [b[0]['score'] for b in gscos]

Question: Construct a proof of the following theorem:

Input: Theorem: The sum of two odd numbers is even.

Answer: Proof: Let's assume the theorem is true. We can take any two odd numbers and add them together to get a sum. This sum, by definition, is also odd, meaning that the sum of two odd numbers must also be even.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:39<00:00,  1.22s/it]


In [11]:
mean([1 if goodscos[i] > badscos[i] else 0 for i in range(len(goodscos))])

0.63